# 1. Set Up the Environment
Download specific libraries to access the model like torch, torchvision and transformers through !pip


In [40]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
!pip install transformers scikit-learn pillow pandas numpy
!pip install opencv-python

Looking in indexes: https://download.pytorch.org/whl/cu128


# 2. Import Libraries
Import necessary libraries to test the model without any issues

In [41]:
import torch # Pytorch to run ViT model
import torch.nn as nn # Pytorch neural network libraries
from torchvision import datasets, transforms # Load the dataset
from torch.utils.data import DataLoader, Dataset # Load the dataset
from torch.optim import AdamW # Optimizer
import torch.nn.functional as F # Provide functions like softmax to get the class probabilities
from transformers import ViTModel, AutoImageProcessor
import torchvision.models as models
from PIL import Image # Library required to modify image dataset
import pandas as pd # To convert text file into label mapping
import numpy as np # Matrix operation
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Cuda available: {torch.cuda.is_available()}")

Cuda available: True


# 3. Install gdown and download folders
Download weights, labels and test folder from google drive through gdown


In [42]:
!pip install -q gdown
import gdown # Library to download file from google drive

model_folder_id = '1eoFx15MLxDo-8Kd-JvIcVLazSVf2j0lq' # Google drive folder id
gdown.download_folder(id=model_folder_id, output="model", quiet=False, use_cookies=False) # Download model weights and labels.txt

Retrieving folder contents


Processing file 1BYkfKnKRW7R5efKWTbK97Ie-nPIIAaS1 vit_full_weights_new2_10.pth
Processing file 1p8KG38pMPPImepVCsK2fCLhHljJ79fj- labels.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1BYkfKnKRW7R5efKWTbK97Ie-nPIIAaS1
From (redirected): https://drive.google.com/uc?id=1BYkfKnKRW7R5efKWTbK97Ie-nPIIAaS1&confirm=t&uuid=490ea331-9ba8-47f7-9c9f-4d11546ea566
To: C:\Users\aloys\PycharmProjects\NAIC Competition\model\vit_full_weights_new2_10.pth
100%|██████████| 347M/347M [00:16<00:00, 21.4MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1p8KG38pMPPImepVCsK2fCLhHljJ79fj-
To: C:\Users\aloys\PycharmProjects\NAIC Competition\model\labels.txt
100%|██████████| 132/132 [00:00<00:00, 381kB/s]
Download completed


['model\\vit_full_weights_new2_10.pth', 'model\\labels.txt']

In [43]:
test_folder_id = '1LZIhiV9l82W4fNpfaBHyGvh8XBX6SoU2' # Google drive testing folder id
gdown.download_folder(id=test_folder_id, output="test_folder", quiet=False, use_cookies=False) # Download testing images

Retrieving folder contents


Processing file 12ufiwhl6whVp33F05RLiDRieRSN13XlP test_labels.txt
Retrieving folder 1qO-cQtRdp-hLu6LMQ68OEv0zA791puJO test_images
Processing file 1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH Kek Lapis.jpg
Processing file 17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB- Kuih Seri Muka.png
Processing file 1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM Kuih Lapis.jpg
Processing file 1KmI2JRpz4DnCAmPbCWoK31BJBH4ax8i7 Kuih Ubi Kayu.jpg
Processing file 1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4 Kuih Kaswi Pandan.jpg
Processing file 1MEhhjAoBK66wtj6Xs1orsrA2vR9qLjhR Kuih Talam.jpg
Processing file 11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6 Kuih Ketayap.jpg
Processing file 1QFZdtM4ok-vH0kPgA7gdZQzT1aSppWcF Onde-Onde.jpg


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=12ufiwhl6whVp33F05RLiDRieRSN13XlP
To: C:\Users\aloys\PycharmProjects\NAIC Competition\test_folder\test_labels.txt
100%|██████████| 283/283 [00:00<00:00, 448kB/s]
Downloading...
From: https://drive.google.com/uc?id=1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH
To: C:\Users\aloys\PycharmProjects\NAIC Competition\test_folder\test_images\Kek Lapis.jpg
100%|██████████| 1.27M/1.27M [00:00<00:00, 17.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB-
To: C:\Users\aloys\PycharmProjects\NAIC Competition\test_folder\test_images\Kuih Seri Muka.png
100%|██████████| 1.04M/1.04M [00:00<00:00, 15.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM
To: C:\Users\aloys\PycharmProjects\NAIC Competition\test_folder\test_images\Kuih Lapis.jpg
100%|██████████| 56.6k/56.6k [00:00<00:00

['test_folder\\test_labels.txt',
 'test_folder\\test_images\\Kek Lapis.jpg',
 'test_folder\\test_images\\Kuih Seri Muka.png',
 'test_folder\\test_images\\Kuih Lapis.jpg',
 'test_folder\\test_images\\Kuih Ubi Kayu.jpg',
 'test_folder\\test_images\\Kuih Kaswi Pandan.jpg',
 'test_folder\\test_images\\Kuih Talam.jpg',
 'test_folder\\test_images\\Kuih Ketayap.jpg',
 'test_folder\\test_images\\Onde-Onde.jpg']

# 4. Load ViT Model
Setting up ViT transformers by adjusting its inner architecture and initialise it

In [44]:
class ViTForClassification(nn.Module): # Define the Vision Transformers Model, creating a framework to create ViT model based on its adjustments
    def __init__(self, vit_model, num_classes=8):
        super().__init__()
        self.vit = vit_model
        self.classifier = nn.Sequential(
            nn.Linear(self.vit.config.hidden_size, 512), # Fully Connected Network have 512 neurons
            nn.BatchNorm1d(512),
            nn.ReLU(), # ReLu activation function
            nn.Dropout(0.3), # Dropout to randomly delete neurons to reduce overfitting
            nn.Linear(512, num_classes), # Fully Connected Network from 512 neurons map to 8 classes
        )

    def forward(self, x):
        outputs = self.vit(pixel_values=x)
        cls_token = outputs.last_hidden_state[:, 0, :] # Modify the token from the transformers
        return self.classifier(cls_token)

def get_vit_model(num_classes):
    vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224') # Get the base pretrained ViT model from google
    return ViTForClassification(vit_model, num_classes) # Match the vit_model output to number of classes

class ViTEnsembleModel(nn.Module):
    def __init__(self, vit_path):
        super().__init__()
        self.softmax = nn.Softmax(dim=0) # Provide softmax function

        self.vit = get_vit_model(8) # Output vit model final layer to 8 classes
        self.vit.load_state_dict(torch.load(vit_path, map_location=device)) # Load the ViT model weights
        self.vit = self.vit.to(device) # Attach vit model to a device(either cpu or gpu)
        self.vit.eval() # Evaluate the vit model, which is to run in testing mode.

    def forward(self, x):
        with torch.no_grad():
            vit_outputs = self.vit(x) # Forward propagation from fully connected network with 512 neurons to final 8 classes

        return vit_outputs

In [46]:
print("Optimizing VIT weights...")
vit_path = 'model/vit_full_weights_new2_10.pth' # Set model weight path
vit = ViTEnsembleModel(vit_path).to(device) # Create ViT model based on ViTEnsembleModel class just now
optimizer = AdamW(vit.parameters(), lr=3e-5) # set optimizer and adjust the learning rate to 0.00003

Optimizing VIT weights...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 5. Load labels.txt

In [47]:
## Change this to where your labels.txt is
labels_filename = 'model/labels.txt'   ##changes here in v2 to point to /content/model
labels = {}
with open(labels_filename, 'r') as f:
    for line in f:
        if line.strip():
            idx, label = line.strip().split(': ', 1)
            # idx only include the value before :, like 0: Kuih Talam split into 0
            # label only include the value after :, like 0: Kuih Talam split into Kuih Talam
            labels[int(idx)] = label # exp: labels[0] = Kuih Talam

label_to_class_index = {v: k for k, v in labels.items()}  # {class_name: index} # Use list function to map label into the index, like 0: Kuih Talam
index_to_label = {k: v for k, v in labels.items()}        # {index: class_name} # Use list function to map index to label, like Kuih Talam :0
print(f"Loaded {len(labels)} classes:")
for idx, label in labels.items():
    print(f"  {idx}: {label}")
print(labels)

Loaded 8 classes:
  0: Kek Lapis
  1: Kuih Kaswi Pandan
  2: Kuih Ketayap
  3: Kuih Lapis
  4: Kuih Seri Muka
  5: Kuih Talam
  6: Kuih Ubi Kayu
  7: Onde-Onde
{0: 'Kek Lapis', 1: 'Kuih Kaswi Pandan', 2: 'Kuih Ketayap', 3: 'Kuih Lapis', 4: 'Kuih Seri Muka', 5: 'Kuih Talam', 6: 'Kuih Ubi Kayu', 7: 'Onde-Onde'}


# 6a. Accessing Testing Directories


In [48]:
# Get list of test images
test_dir = 'test_folder/test_images'  ##changes here in v2 to point to /content/model
test_images = []
for root, _, files in os.walk(test_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            test_images.append(os.path.join(root, file)) # Add the image into the file

test_images.sort()  # Sort to ensure consistent order
print(f"Found {len(test_images)} test images")

Found 8 test images


# 6b. Ensure label correctness

In [49]:
df_test = pd.read_csv('test_folder/test_labels.txt')

# Clean class labels: remove trailing digits (like "Kuih Talam 1"), and strip spaces
df_test['Class'] = df_test['Class'].str.replace(r'\s*\d+$', '', regex=True).str.strip()

# Validation
test_class_names = set(df_test['Class'].unique())
for class_name in test_class_names:
    if class_name not in label_to_class_index:
        print(f"Error: Class '{class_name}' in test set is not present in labels.txt")
        raise ValueError("Test set contains unknown classes")
print("All test classes are present in labels.txt")

# Create lookup dict
test_labels = dict(zip(df_test['Filename'].str.strip(), df_test['Class'].str.strip()))
filename_to_class = test_labels.copy()

All test classes are present in labels.txt


# 7. CSV Converter
Convert test folder images into csv file


In [50]:
import os
import glob
import csv
import cv2
import numpy as np
import re
from pathlib import Path

testing_image_folder = Path("test_folder/test_images")
testing_csv_file = "test.csv"
image_size = (224, 224)

def clean_label_from_filename(filename):
    """
    Example: 'Kek Lapis 2.jpg' → 'Kek Lapis'
    """
    name = os.path.splitext(filename)[0]              # remove .jpg
    label = re.sub(r'\s*\d+$', '', name).strip()      # remove trailing number
    return label


def write_dataset_to_csv(image_folder, csv_file):
    # Include different type of images like jpg, jpeg and png
    # If there's different type of image file type please help add it below the image_files based on the pattern. Thanks!
    image_files = list(image_folder.glob("*.jpg")) + \
                  list(image_folder.glob("*.jpeg")) + \
                  list(image_folder.glob("*.png")) + \
                  list(image_folder.glob("*.WebP"))

    with open(csv_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["filename", "label", "pixels"])  # Added filename column

        for image_file in image_files:
            try:
                image = cv2.imread(image_file)
                if image is None:
                    print(f"Error reading image {image_file}")
                    continue

                image = cv2.resize(image, image_size)
                image_array = np.array(image).flatten()
                image_data_str = ','.join(map(str, image_array))

                filename = os.path.basename(image_file)
                label = clean_label_from_filename(filename)

                csv_writer.writerow([filename, label, image_data_str])  # Store original filename

            except Exception as e:
                print(f"Error processing {image_file}: {e}")

write_dataset_to_csv(testing_image_folder, testing_csv_file)
print(f"CSV file '{testing_csv_file}' created successfully.")

df = pd.read_csv('test.csv')
print(f"CSV contains {len(df)} rows")
print("Missing images:", set(os.path.basename(p) for p in test_images) - set(df['filename']))


CSV file 'test.csv' created successfully.
CSV contains 8 rows
Missing images: set()


# 8. Data Loader
Load dataset from csv file

In [51]:
class DataSetLoader(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

        # Check for required columns
        if 'pixels' not in self.data.columns or 'label' not in self.data.columns:
            raise ValueError("CSV must contain 'pixels' and 'label' columns.")

        # Drop rows with missing values
        self.data.dropna(subset=['pixels', 'label'], inplace=True)

        # Create label map
        unique_labels = sorted(self.data['label'].unique())
        self.label_map = {label: idx for idx, label in enumerate(unique_labels)}
        self.num_classes = len(self.label_map)

        # Store original filenames if available (optional)
        if 'filename' in self.data.columns:
            self.filenames = self.data['filename'].values
        else:
            self.filenames = None

        print(f"[INFO] Loaded {len(self.data)} samples with {self.num_classes} unique classes.")
        print(f"[INFO] Label map: {self.label_map}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      # Decode pixel string into image array
      pixel_str = self.data.iloc[idx]['pixels']
      image_flat = np.fromstring(pixel_str, sep=',', dtype=np.uint8)

      try:
          image = image_flat.reshape((224, 224, 3))
      except ValueError:
          raise ValueError(f"Image at index {idx} could not be reshaped. Array shape: {image_flat.shape}")

      image = Image.fromarray(image)

      # Get the string label and convert to integer using label_map
      str_label = self.data.iloc[idx]['label']
      label = self.label_map[str_label]  # This gives us the integer label
      label = torch.tensor(label, dtype=torch.long)

      # Get filename (either from CSV or generate it)
      if self.filenames is not None:
          filename = self.filenames[idx]
      else:
          filename = f"{str_label}.jpg"

      if self.transform:
          image = self.transform(image)

      return image, label, filename

    def get_class_names(self):
        """Returns list of class names in order of their indices"""
        return sorted(self.label_map.keys(), key=lambda x: self.label_map[x])



# 9. Running Predictions
Testing phase

In [52]:
predictions = []
processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224')

# Data preprocessing
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

test_dataset = DataSetLoader('test.csv', test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(test_dataset.label_map)


index_to_label = {v: k for k, v in test_dataset.label_map.items()}
label_to_class_index = {v: k for k, v in index_to_label.items()}
print("Index to label: ", index_to_label)
print("Label to index: ", label_to_class_index)
print("Full label map:", test_dataset.label_map)

with torch.no_grad():
    for images, labels, filenames in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = vit(images)
        probabilities = F.softmax(outputs, dim=1)
        predicted_indices = torch.argmax(probabilities, dim=1)

        for i in range(len(images)):
            filename = filenames[i]
            pred_idx = predicted_indices[i].item()
            actual_idx = labels[i].item()
            pred_label = index_to_label.get(pred_idx, f"unknown_class_{pred_idx}")
            actual_label = index_to_label[actual_idx]
            class_probs = probabilities[i].cpu().numpy()

            # Store prediction result
            predictions.append({
                'image': filename,
                'predicted_class_index': pred_idx,
                'predicted_label': pred_label,
                'class_probabilities': class_probs.tolist()  # convert to list for JSON-safe export
            })

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


[INFO] Loaded 8 samples with 8 unique classes.
[INFO] Label map: {'Kek Lapis': 0, 'Kuih Kaswi Pandan': 1, 'Kuih Ketayap': 2, 'Kuih Lapis': 3, 'Kuih Seri Muka': 4, 'Kuih Talam': 5, 'Kuih Ubi Kayu': 6, 'Onde-Onde': 7}
{'Kek Lapis': 0, 'Kuih Kaswi Pandan': 1, 'Kuih Ketayap': 2, 'Kuih Lapis': 3, 'Kuih Seri Muka': 4, 'Kuih Talam': 5, 'Kuih Ubi Kayu': 6, 'Onde-Onde': 7}
Index to label:  {0: 'Kek Lapis', 1: 'Kuih Kaswi Pandan', 2: 'Kuih Ketayap', 3: 'Kuih Lapis', 4: 'Kuih Seri Muka', 5: 'Kuih Talam', 6: 'Kuih Ubi Kayu', 7: 'Onde-Onde'}
Label to index:  {'Kek Lapis': 0, 'Kuih Kaswi Pandan': 1, 'Kuih Ketayap': 2, 'Kuih Lapis': 3, 'Kuih Seri Muka': 4, 'Kuih Talam': 5, 'Kuih Ubi Kayu': 6, 'Onde-Onde': 7}
Full label map: {'Kek Lapis': 0, 'Kuih Kaswi Pandan': 1, 'Kuih Ketayap': 2, 'Kuih Lapis': 3, 'Kuih Seri Muka': 4, 'Kuih Talam': 5, 'Kuih Ubi Kayu': 6, 'Onde-Onde': 7}


# 10. Creating Outputs
Printing output using pandas library

In [53]:
results_df = pd.DataFrame(predictions)
display(results_df)

,image,predicted_class_index,predicted_label,class_probabilities
0,Kek Lapis.jpg,0,Kek Lapis,"[0.9200485348701477, 0.010303686372935772, 0.0..."
1,Kuih Kaswi Pandan.jpg,1,Kuih Kaswi Pandan,"[0.01865220069885254, 0.8201457262039185, 0.01..."
2,Kuih Ketayap.jpg,2,Kuih Ketayap,"[0.011439827270805836, 0.01115582324564457, 0...."
3,Kuih Lapis.jpg,3,Kuih Lapis,"[0.007907097227871418, 0.009971368126571178, 0..."
4,Kuih Talam.jpg,5,Kuih Talam,"[0.00497874990105629, 0.006190862040966749, 0...."
5,Kuih Ubi Kayu.jpg,6,Kuih Ubi Kayu,"[0.008483816869556904, 0.010306216776371002, 0..."
6,Onde-Onde.jpg,7,Onde-Onde,"[0.011046474799513817, 0.011978148482739925, 0..."
7,Kuih Seri Muka.png,4,Kuih Seri Muka,"[0.019940165802836418, 0.0636836513876915, 0.0..."


# 11. Metrics Computation
Calculate the accuracy, precision, recall, F1 and ROC-AUC of the output result

In [54]:
import pandas as pd
import re
import os

# Re-read and preprocess test_labels.txt
df_test = pd.read_csv('test_folder/test_labels.txt')
df_test['Class'] = df_test['Class'].str.replace(r'\s*\d+$', '', regex=True).str.strip()
df_test['Filename'] = df_test['Filename'].str.strip()

# Build filename_to_class from test labels
filename_to_class = dict(zip(df_test['Filename'], df_test['Class']))

# Assume labels is already defined
label_to_class_index = {v.strip(): k for k, v in index_to_label.items()}

# ✅ Updated function: extract class name from filename
def extract_class_from_filename(filename):
    name_part = os.path.splitext(filename)[0]  # e.g., 'Kek Lapis 1'
    name_part = re.sub(r'\s*\d+$', '', name_part)  # Remove trailing numbers
    return name_part.strip()

# Create true_class_index from only the filenames that were processed
true_class_index = []

for filename in results_df['image']:
    filename = filename.strip()
    class_name = extract_class_from_filename(filename)

    if class_name not in label_to_class_index:
        raise ValueError(f"Unknown class '{class_name}' extracted from filename '{filename}'")

    true_class_index.append(label_to_class_index[class_name])

print("True class index: ", true_class_index)


True class index:  [0, 1, 2, 3, 5, 6, 7, 4]


In [55]:
from sklearn.metrics import (
    classification_report, accuracy_score,
    roc_auc_score, precision_recall_fscore_support
)
from sklearn.preprocessing import label_binarize

results_df['true_class_index'] = true_class_index  ## changes here in v2 - refer to true_class_index
y_true = results_df['true_class_index'].astype(int).values
y_pred = results_df['predicted_class_index'].astype(int).values

print(y_true)
print(y_pred)

## Quick fix for ROC Curve as I only have 3 classes here (DO NOT NEED THIS IF YOU HAVE 8 CLASSES IN YOUR TEST SET)
FULL_NUM_CLASSES = 8  # total number of possible classes

# ✅ Fix: Pad each probability list to 8 elements
def pad_probs(probs, target_len=FULL_NUM_CLASSES):
    probs = np.ravel(probs)  # Ensure 1D
    padded = np.zeros(target_len)
    padded[:len(probs)] = probs  # Assumes order is correct
    return padded

# ✅ Apply padding safely
y_probs_padded = np.array([pad_probs(p) for p in results_df['class_probabilities']])
y_probs = y_probs_padded  # Now it's safe


[0 1 2 3 5 6 7 4]
[0 1 2 3 5 6 7 4]


In [56]:
# Number of classes
n_classes = FULL_NUM_CLASSES
class_names = list(range(FULL_NUM_CLASSES))

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")

# Precision, Recall, F1 per class & macro
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_names, average=None)
macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

print("\n📊 Per-class metrics:")
for i, cls in enumerate(class_names):
    print(f"Class {cls}: Precision={prec[i]:.4f}, Recall={rec[i]:.4f}, F1={f1[i]:.4f}")

print(f"\n📦 Macro Precision: {macro_prec:.4f}, Macro Recall: {macro_rec:.4f}, Macro F1: {macro_f1:.4f}")

# ROC AUC (requires binarized labels)
y_true_bin = label_binarize(y_true, classes=class_names)

# ROC AUC per class and macro
try:
    auc_per_class = roc_auc_score(y_true_bin, y_probs, average=None, multi_class='ovr')
    auc_macro = roc_auc_score(y_true_bin, y_probs, average='macro', multi_class='ovr')

    print("\n🎯 ROC AUC per class:")
    for i, cls in enumerate(class_names):
        print(f"Class {cls}: AUC = {auc_per_class[i]:.4f}")

    print(f"\n🌐 Macro ROC AUC: {auc_macro:.4f}")

except Exception as e:
    print(f"⚠️ ROC AUC could not be computed: {e}")



✅ Accuracy: 1.0000

📊 Per-class metrics:
Class 0: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 1: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 2: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 3: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 4: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 5: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 6: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 7: Precision=1.0000, Recall=1.0000, F1=1.0000

📦 Macro Precision: 1.0000, Macro Recall: 1.0000, Macro F1: 1.0000

🎯 ROC AUC per class:
Class 0: AUC = 1.0000
Class 1: AUC = 1.0000
Class 2: AUC = 1.0000
Class 3: AUC = 1.0000
Class 4: AUC = 1.0000
Class 5: AUC = 1.0000
Class 6: AUC = 1.0000
Class 7: AUC = 1.0000

🌐 Macro ROC AUC: 1.0000
